# Run a basic Hermes Hartree-Fock energy calculation on ligand conformers from Auto3d

See [the tutorial](/Tutorials/auto3d_conformer_tautomer_generation.ipynb).

In [ ]:
import json
import os
import sys
import tarfile

from pdbtools import *
import requests
from datetime import datetime
from pathlib import Path
import py3Dmol

import rush

# Define our project information
EXPERIMENT = "tengu-py-auto3d_hermes"
LIGAND_SMILES = "CC(=O)OC1=CC=CC=C1C(=O)O "
LIGAND = "ASPRIN"
TAGS = ["qdx", EXPERIMENT, LIGAND]

# |hide_line
WORK_DIR = Path.home() / "qdx" / EXPERIMENT

# |hide_line
if WORK_DIR.exists():
# |hide_line
    client = rush.Provider(workspace=WORK_DIR)
# |hide_line
    await client.nuke(remote=False)

# |hide_line
os.makedirs(WORK_DIR, exist_ok=True)
# |hide_line
os.makedirs(WORK_DIR / ".rush", exist_ok=True)
# |hide_line
import sys

# |hide_line
os.chdir(WORK_DIR)

# Get our client, for calling modules and using the rush API
# - edit in your access token and API URL or set the RUSH_TOKEN and RUSH_URL environment variables
client = await rush.build_provider_with_functions(
    access_token=os.getenv("RUSH_TOKEN"),
    url=os.getenv("RUSH_URL"),
    batch_tags=TAGS,
)

# |hide_line
client = await rush.build_provider_with_functions(batch_tags=TAGS, restore_by_default=True)

# |hide_line
client.workspace = WORK_DIR
# |hide_line
client.config_dir = WORK_DIR / ".rush"

# Preparation
ligand_path = client.workspace / "aspirin.smiles"
with open(ligand_path, "w") as ligand_file:
    print(f"{LIGAND_SMILES} {LIGAND_SMILES}", file=ligand_file)

(conformer_sdf, conformer_qdxf) = await client.auto3d(
    ligand_path, "smi", {"k": 5, "use_gpu": True}, resources={"gpus": 1, "storage": "5", "storage_units": "GB"},
    target="NIX_SSH_3"
)

print(f"{datetime.now().time()} | Running ligand preparation!")

try:
    await conformer_sdf.download(filename="01_prepared_ligand.sdf")
except FileExistsError:
    # we will raise an error if you try to overwrite an existing file; you can
    # force the file to overwrite by passing an absolute filepath instead
    pass

# Visualize the prepared ligand to spot check any issues
view = py3Dmol.view()
with open(client.workspace / "objects" / "01_prepared_ligand.sdf", "r") as f:
    view.addModel(f.read(), "sdf")
    view.setStyle({"stick": {}})
    view.zoomTo()
    view.show()

# Quantum energy calculation
HERMES_RESOURCES = {
        "gpus": 1,
        "storage": 100,
        "storage_units": "MB",
        "walltime": 60,
}

(conformer,) = await client.pick_conformer(conformer_qdxf, 0)
conformer_out = await conformer.get()

(hermes_energy,) = await client.hermes_energy(
    conformer,
    {},
    {
        "method": "RestrictedRIMP2",
        "basis": "cc-pVDZ",
        "aux_basis": "cc-pVDZ-RIFIT",
        "force_cartesian_basis_sets": True,
        "standard_orientation": "FullSystem",
    },
    {
    "scf": {
        "max_iter": 50,
        "max_diis_history_length": 12,
        "convergence_metric": "DIIS",
        "convergence_threshold": 0.000001,
        "density_threshold": 1e-10,
        "density_basis_set_projection_fallback_enabled": True,
        },
    "frag": {
        "cutoffs": {"dimer": 22},
        "cutoff_type": "Centroid",
        "level": "Dimer",
        "reference_fragment": len(conformer_out["topology"]["fragments"])
        - 1,
        },
    },
    target="GADI",
    resources=HERMES_RESOURCES,
)

await hermes_energy.get()

Exception: Unknown kind @